# 載入所需套件

In [ ]:
!pip install unsloth vllm bitsandbytes peft transformers datasets trl accelerate

In [ ]:
!pip install transformers accelerate --upgrade
!pip install --upgrade bitsandbytes
!pip install --upgrade einops

In [ ]:
import re
import pandas as pd
from datasets import Dataset
from tqdm import tqdm

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import GRPOConfig, GRPOTrainer
from peft import LoraConfig, get_peft_model, PeftModel
from transformers import TrainerCallback

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# 資料前處理

In [ ]:
# model_name = "Qwen/Qwen2.5-3B-Instruct"

# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.model_max_length = 512

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     trust_remote_code=True,
#     device_map="auto"
# )

# print("max_seq_length set to:", tokenizer.model_max_length)

model_name = "Qwen/Qwen2.5-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.model_max_length = 512

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16,       
    low_cpu_mem_usage=True,          
    device_map="auto"                
)

print("max_seq_length set to:", tokenizer.model_max_length)


In [ ]:
df = pd.read_csv("train-reasoning-v2.csv")
df = df.fillna("")

# 建立 prompt（題幹 + 選項 + 格式說明）
def build_prompt(row):  
    q = str(row["題目"]).strip()
    opts = [str(row[f"選項{c}"]).strip() for c in ["A", "B", "C", "D"]]
    return (
        "請閱讀以下選擇題，並輸出解題步驟與正確答案（使用指定格式）：\n"
        + q + "\n"
        + "\n".join([f"{c}. {opt}" for c, opt in zip(["A", "B", "C", "D"], opts)])
        + "\n請依照以下格式輸出：\n<reasoning>...</reasoning>\n<answer>...</answer>"
    )

# 清理推理輸出內容
def clean_reasoning(text, question):
    text = str(text)
    question = str(question)

    # 去除無用 tag
    tags_to_remove = ["<question>", "</question>", "<think>", "</think>",
                      "<reflection>", "</reflection>", "<adjustment>", "</adjustment>",
                      "<output>", "</output>"]
    for tag in tags_to_remove:
        text = text.replace(tag, "")

    # 移除題目內容本身（避免出現在 reasoning 裡）
    text = text.replace(question.strip(), "").strip()

    # 抽出 <reasoning> 段落（第一段為主）
    match = re.search(r"<reasoning>([\s\S]*?)</reasoning>", text)
    reasoning_clean = match.group(1).strip() if match else text.strip()

    return reasoning_clean


# 建立 output（模型訓練目標）
def build_output(row):
    reasoning = clean_reasoning(row["推理正確答案"], row["題目"])
    answer = str(row["正確答案"]).strip()[0]  # 只保留 A/B/C/D
    return f"<reasoning>\n{reasoning}\n</reasoning>\n<answer>\n{answer}\n</answer>"

# 套用 & 轉為 Dataset 格式
df["prompt"] = df.apply(build_prompt, axis=1)
df["output"] = df.apply(build_output, axis=1)

df = df.reset_index(drop=True)
train_dataset = Dataset.from_pandas(df[["prompt", "output"]])

In [ ]:
print(train_dataset[0]["prompt"])

In [ ]:
print(train_dataset[0]["output"])

# 訓練

In [ ]:
# 「產生 reward function」，會根據你提供的正確答案列表 solutions，以及最短推理長度 min_len
def make_reasoning_answer_reward(solutions, min_len=100):
    p_reason = re.compile(r"<reasoning>\s*([\s\S]*?)\s*</reasoning>", re.I)
    p_answer = re.compile(r"<answer>\s*([A-D])\s*</answer>", re.I)

    def _reward(completions, **kwargs):
        scores = []
        for comp, sol in zip(completions, solutions):
            if isinstance(comp, str):
                text = comp
            elif isinstance(comp, dict):
                text = comp.get("content", "")
            elif isinstance(comp, list) and isinstance(comp[0], dict):
                text = comp[0].get("content", "")
            else:
                text = ""

            m_r = p_reason.search(text)
            reasoning_ok = bool(m_r and len(m_r.group(1).strip()) >= min_len)

            m_a = p_answer.search(text)
            pred = m_a.group(1).upper() if m_a else ""
            answer_ok = pred == sol

            scores.append(1.0 if reasoning_ok and answer_ok else 0.0)
        return scores

    return _reward

In [ ]:
solutions = df["正確答案"].str.strip().str.upper().str[0].tolist()
reasoning_answer_reward = make_reasoning_answer_reward(solutions)

In [ ]:
# 資料前處理
def make_preprocess(tokenizer):
    def preprocess(ex):
        tok = tokenizer(ex["prompt"], truncation=True, max_length=128, padding="max_length")
        out = tokenizer(ex["output"], truncation=True, max_length=64, padding="max_length")
        input_ids = out["input_ids"]

        # 確保有非 -100 的 token
        labels = [(-100 if token == tokenizer.pad_token_id else token) for token in input_ids]
        tok["labels"] = labels
        return tok
    return preprocess

preprocess = make_preprocess(tokenizer)
train_dataset = train_dataset.map(preprocess, batched=True)

In [ ]:
def print_sample_output(model, tokenizer, step=0):
    prompt = "請閱讀以下選擇題，並輸出解題步驟與正確答案（使用指定格式）：\n" \
             "中國的「天網工程」主要目的是什麼？\n" \
             "A. 推廣旅遊景點\nB. 透過攝影機和AI進行全民監控\nC. 改善氣象預報\nD. 提升城市交通效率\n" \
             "請依照以下格式輸出：\n<reasoning>...</reasoning>\n<answer>...</answer>"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output_ids = model.generate(
        **inputs,
        max_new_tokens=64,
        do_sample=True,
        top_k=50,
        temperature=0.7
    )
    print(f"\n[🧠 Step {step}] === Sample Output ===")
    print(tokenizer.decode(output_ids[0], skip_special_tokens=True))
    print("=" * 50)

In [ ]:
class SampleGenerationCallback(TrainerCallback):
    def __init__(self, tokenizer, every_n_steps=50):
        self.tokenizer = tokenizer
        self.every_n_steps = every_n_steps

    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % self.every_n_steps == 0:
            print_sample_output(kwargs["model"], self.tokenizer, step=state.global_step)

In [ ]:
# LoRA 微調設定與套用
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, peft_config)

In [ ]:
grpo_config = GRPOConfig(
    learning_rate=1e-5,
    remove_unused_columns=False,
    gradient_accumulation_steps=16,
    num_train_epochs=5,
    bf16=False,
    fp16=True,
    max_completion_length=64,
    num_generations=4,
    max_prompt_length=128,
    report_to=["tensorboard"],
    logging_steps=100,
    push_to_hub=False,  
    save_strategy="steps",
    save_steps=100,
)

trainer = GRPOTrainer(
    model=model,
    train_dataset=train_dataset,
    args=grpo_config,
    reward_funcs=[reasoning_answer_reward],
    callbacks=[SampleGenerationCallback(tokenizer, every_n_steps=50)]
)

trainer.train()

# 推理(原本)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-3B-Instruct")
merged_model = PeftModel.from_pretrained(base_model, "./grpo-trained-model")
merged_model = merged_model.merge_and_unload()

# 儲存完整模型（含 config.json & pytorch_model.bin）
merged_model.save_pretrained("./merged-model")
tokenizer.save_pretrained("./merged-model")

In [ ]:
!pip install huggingface_hub

from huggingface_hub import login

login()  # 執行後貼上你的 token

In [ ]:
from huggingface_hub import upload_folder, create_repo

repo_id = "chenyu313707056/GRPO313707056"
local_model_path = "./merged-model"  # ⬅️ 請改為你模型存放的資料夾

create_repo(repo_id, exist_ok=True)

upload_folder(
    repo_id=repo_id,
    folder_path=local_model_path,
    path_in_repo="",
    commit_message="Upload full GRPO model",
)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "./merged-model"  # ⬅️ 請替換為你儲存模型的資料夾路徑

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
model = model.eval().cuda()


In [ ]:
import pandas as pd

# 讀取 test 檔案（你剛剛上傳的）
test_df = pd.read_csv("test-check-v2.csv")

# 建立與訓練格式一致的 prompt
def format_prompt(row):
    q = row["Question"]
    options = [row[f"Option {c}"] for c in ["A", "B", "C", "D"]]
    prompt = (
        q + "\n" +
        "\n".join([f"{c}. {opt}" for c, opt in zip(["A", "B", "C", "D"], options)]) +
        "\n請只輸出答案選項（A/B/C/D）："
    )
    return prompt

test_df["prompt"] = test_df.apply(format_prompt, axis=1)


In [ ]:
from tqdm import tqdm

def generate_answer(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=4,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

# 對全部 prompt 做推論
test_df["prediction"] = [generate_answer(p) for p in tqdm(test_df["prompt"])]


In [ ]:
import re

# 從模型輸出中萃取 A/B/C/D
def extract_choice(text):
    match = re.search(r"\b([A-D])\b", text.strip().upper())
    return match.group(1) if match else "A"  # 若沒抓到預設 A

test_df["Answer"] = test_df["prediction"].apply(extract_choice)
test_df["ID"] = range(1, len(test_df) + 1)

# 儲存為 submission.csv
submission = test_df[["ID", "Answer"]]
submission.to_csv("submission.csv", index=False)


# 推理(改)

In [ ]:
# 載入模型 + tokenizer（包含訓練好的 LoRA）
base_model = "Qwen/Qwen2.5-3B-Instruct"
lora_path = "trainer_output/checkpoint-1000"   # 替換 LoRA 模型路徑

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16      
)
model = PeftModel.from_pretrained(model, lora_path)
model.eval()

# 載入測試資料
df = pd.read_csv("test-check-v2.csv").fillna("")

# 匯出答案列表
pred_answers = []

for i, row in tqdm(df.iterrows(), total=len(df), desc="Predicting"):
    q = row["Question"]
    opts = [row[f"Option {c}"] for c in ["A", "B", "C", "D"]]
    prompt = (
        "請閱讀以下選擇題，並輸出解題步驟與正確答案（使用指定格式）：\n"
        "\n請務必輸出以下格式，不可省略 answer 標籤：\n"
        + q + "\n"
        + "\n".join([f"{c}. {opt}" for c, opt in zip(["A", "B", "C", "D"], opts)])
        + "\n請依照以下格式輸出：\n<reasoning>...</reasoning>\n<answer>...</answer>"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=160,
            do_sample=True,
            top_k=50,
            temperature=0.7,
        )
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # 用 regex 抽出答案（A/B/C/D）
    match = re.search(r"<answer>\s*([A-D])\s*</answer>", output_text)
    ans = match.group(1) if match else "?"

    pred_answers.append(ans)

df["predict_answer"] = pred_answers
df = df[["ID", "predict_answer"]]
df.to_csv("submission.csv", index=False)
print("預測完成，已儲存為 submission.csv")